In [1]:
!pip install apache-airflow openai

^C
Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/45.4 kB ? eta -:--:--
     ---------------------------------- --- 41.0/45.4 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 45.4/45.4 kB 450.4 kB/s eta 0:00:00
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.4 MB 17.2 MB/s eta 0:00:01
   --- ------------------------------------ 1.3/13.4 MB 16.7 MB/s eta 0:00:01
   ------ --------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-instrumentation 0.50b0 requires opentelemetry-semantic-conventions==0.50b0, but you have opentelemetry-semantic-conventions 0.51b0 which is incompatible.
opentelemetry-instrumentation-asgi 0.50b0 requires opentelemetry-semantic-conventions==0.50b0, but you have opentelemetry-semantic-conventions 0.51b0 which is incompatible.
opentelemetry-instrumentation-fastapi 0.50b0 requires opentelemetry-semantic-conventions==0.50b0, but you have opentelemetry-semantic-conventions 0.51b0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
from openai import OpenAI
import os

# Set your OpenAI API key (or use Airflow's connection management)
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-api-key-here")
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
openai = OpenAI()

# Define the function to call OpenAI's API
def call_openai():
    #openai.api_key = OPENAI_API_KEY

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are an AI assistant."},
                  {"role": "user", "content": "Generate a fun fact about space."}]
    )

    fact = response["choices"][0]["message"]["content"]
    print(f"OpenAI Response: {fact}")

# Define default arguments for the DAG
default_args = {
    "owner": "airflow",
    "depends_on_past": False,
    "start_date": datetime(2024, 1, 1),
    "retries": 1,
    "retry_delay": timedelta(minutes=5),
}

# Define the DAG
with DAG(
    "openai_text_generation",
    default_args=default_args,
    description="A simple DAG to generate text using OpenAI",
    schedule_interval=timedelta(days=1),  # Runs daily
    catchup=False,
) as dag:

    generate_text_task = PythonOperator(
        task_id="generate_openai_text",
        python_callable=call_openai,
    )

    generate_text_task


c:\Users\e022280\VScode\LLMEng\LLMEng_venv\Lib\site-packages\airflow\__init__.py:36: RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.
  warnings.warn(
OSError while attempting to symlink the latest log directory


C:\Users\e022280\AppData\Local\Temp\ipykernel_54556\876233828.py:38 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.